In [95]:
import pandas as pd
import re
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.stem.snowball import SnowballStemmer

In [96]:
df_amazon = pd.read_csv('../data/raw/Reviews_amazon.csv',index_col=0)
df_milanuncios = pd.read_csv('../data/raw/Reviews_MilAnuncios.csv',index_col=0)
df_vinted = pd.read_csv('../data/raw/Reviews_Vinted.csv',index_col=0)
df_wallapop = pd.read_csv('../data/raw/Reviews_Wallapop.csv',index_col=0)
df_booking = pd.read_csv('../data/raw/Reviews_Booking.csv',index_col=0)
df_trip = pd.read_csv('../data/raw/Reviews_Trip.csv',index_col=0)
df_mytrip = pd.read_csv('../data/raw/Reviews_Mytrip.csv', index_col=0)
df_kiwi = pd.read_csv('../data/raw/Reviews_Kiwi.csv', index_col=0)
df_edreams = pd.read_csv('../data/raw/Reviews_Edreams.csv', index_col=0)
df_myreviews = pd.read_csv('../data/raw/MyReviews.csv',index_col=0)
df_opodo = pd.read_csv('../data/raw/Reviews_Opodo.csv', index_col=0)
df_mediamarkt = pd.read_csv('../data/raw/Reviews_MediaMarkt.csv', index_col=0)
df_corte = pd.read_csv('../data/raw/Reviews_ElCorteIngles.csv',index_col=0)
df_pccomp = pd.read_csv('../data/raw/Reviews_PcComponentes.csv', index_col=0)
df_aliexp = pd.read_csv('../data/raw/Reviews_AliExpress.csv', index_col=0)
df_viagogo = pd.read_csv('../data/raw/Reviews_Viagogo.csv', index_col=0)
df_stubhub = pd.read_csv('../data/raw/Reviews_Stubhub.csv', index_col=0)
df_despegar = pd.read_csv('../data/raw/Reviews_Despegar.csv', index_col=0)
df_uber = pd.read_csv('../data/raw/Reviews_Uber.csv',index_col=0)
df_ryanair = pd.read_csv('../data/raw/Reviews_Ryanair.csv',index_col=0)
df_easyjet = pd.read_csv('../data/raw/Reviews_EasyJet.csv',index_col=0)
df_balearia = pd.read_csv('../data/raw/Reviews_Balearia.csv',index_col=0)
df_goldcar = pd.read_csv('../data/raw/Reviews_Goldcar.csv',index_col=0)
df_recordrac = pd.read_csv('../data/raw/Reviews_RecordRAC.csv',index_col=0)
df_okmob = pd.read_csv('../data/raw/Reviews_OkMob.csv',index_col=0)
df_create = pd.read_csv('../data/raw/Reviews_Create.csv',index_col=0)
df_clearpay = pd.read_csv('../data/raw/Reviews_Clearpay.csv',index_col=0)
df_travelgenio = pd.read_csv('../data/raw/Reviews_Travelgenio.csv',index_col=0)
df_travel2be = pd.read_csv('../data/raw/Reviews_Travel2be.csv',index_col=0)
df_bnext = pd.read_csv('../data/raw/Reviews_Bnext.csv',index_col=0)
df_carjet = pd.read_csv('../data/raw/Reviews_Carjet.csv',index_col=0)

df = pd.concat([df_amazon,df_milanuncios,df_vinted,df_wallapop,df_booking,\
                df_trip,df_mytrip, df_kiwi, df_myreviews,df_edreams, df_opodo,\
                df_mediamarkt,df_corte,df_pccomp, df_aliexp, df_viagogo, \
                df_stubhub, df_despegar, df_uber, df_ryanair, df_easyjet, \
                df_balearia, df_goldcar, df_recordrac, df_okmob, df_create, \
                df_clearpay, df_travelgenio, df_travel2be, df_bnext, df_carjet], axis=0)
df['Estrellas'].value_counts()

5    42663
1    36314
4     7509
3     3255
2     2608
Name: Estrellas, dtype: int64

In [97]:
df.reset_index(drop=True, inplace=True)

In [98]:
df

,Estrellas,Reseñas
0,5,Fiable con servicio al comprador incomparable....
1,1,Amazon exige que le entregue la firma del vend...
2,1,Hace unos días compré un lote y la…Hace unos d...
3,5,Amazon muy buena experienciaAños haciendo pedi...
4,1,La peor tienda onlineEs un completo asco de ti...
...,...,...
92344,5,Siempre perfectosSiempre perfectos. Y tuve que...
92345,5,Bueno de momento bienBueno de momento bien. a ...
92346,5,Muy fácil realizar la reservaMuy fácil realiza...
92347,5,Fácil cómodo y económicoFácil cómodo y económi...


In [99]:
sentimiento = []

for i in df['Estrellas']:
    if i <= 3:
        sentimiento.append(0)
    else:
        sentimiento.append(1)

df['Sentimiento'] = sentimiento

In [100]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alexc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## Primer preprocesado del texto.

Cuando coge las reseñas, pega el último caracter del título de la reseña con el primero del texto. Por ello, separo, con una expresión regular si hay una minúscula, o un punto, seguido de un caracter en mayúscula por un espacio en blanco

In [101]:
lista_resenas = []
for i in df['Reseñas']:
    resultado = re.sub(r"([a-z\W\d])([A-Z])", r"\1 \2", i)
    lista_resenas.append(resultado)

df['Reseñas'] = lista_resenas


## Segundo preprocesado

Eliminamos puntuación, pasamos todo a minúsculas, eliminamos las palabras comunes o stopwords, y unimos las palabras

In [102]:
stopwords = nltk.corpus.stopwords.words('spanish')
stemmer = SnowballStemmer("spanish")


def preprocess_text(text):
    # Eliminar puntuación
    text = ''.join([c for c in text if c not in punctuation])

    # Convertir a minúsculas el texto
    text = text.lower()

    # Tokenización
    tokens = nltk.word_tokenize(text)

    # # Eliminar stopwords
    # tokens = [word for word in tokens if word not in stopwords]

    #Aplicar un stemmer
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    stemmed_text = " ".join(stemmed_tokens)
    return stemmed_text

df['Texto procesado'] = df['Reseñas'].apply(preprocess_text)


In [103]:
df.tail()

,Estrellas,Reseñas,Sentimiento,Texto procesado
92344,5,Siempre perfectos Siempre perfectos. Y tuve q...,1,siempr perfect siempr perfect y tuv que utiliz...
92345,5,Bueno de momento bien Bueno de momento bien. a...,1,buen de moment bien buen de moment bien a ver ...
92346,5,Muy fácil realizar la reserva Muy fácil realiz...,1,muy facil realiz la reserv muy facil realiz la...
92347,5,Fácil cómodo y económico Fácil cómodo y económ...,1,facil comod y econom facil comod y econom buen...
92348,5,todo perfecto,1,tod perfect


In [104]:
df.to_csv('../data/processed/Reviews_FINAL.csv')